#### Crypto Coin Price Prediction Using Long-Short-Term-Model Algorithm in Neural Networks 
 Developers
+ 214210050 Furkan Demircan
+ 214210056 Eray Elagoz 
---------------------------------------------------------------------------

##### Objective
It aims to leverage machine learning & Deep Learning techniques to develop an accurate and reliable model for predicting and forecasting Bitcoin prices. By utilizing historical data and relevant features, the model provided insights into the potential future trends of Bitcoin, assisting investors, traders, and researchers in making informed decisions in the cryptocurrency market.

##### Libraries

In [ ]:
# Data manipulation and analysis
import numpy as np # linear algebra
import pandas as pd # data processing
import matplotlib.pyplot as plt # data visualization
import yfinance as yf # for coin data

# Main libraries
import datetime as dt # for date and time operations

# Machine learning
from sklearn.preprocessing import MinMaxScaler # for normalization
from sklearn.metrics import mean_squared_error, mean_absolute_error,root_mean_squared_error, mean_absolute_percentage_error # for model evaluation
from sklearn.metrics import f1_score, accuracy_score, classification_report, recall_score, precision_score # for model evaluation

# Deep learning
import tensorflow as tf # For neural networks
from tensorflow.keras.models import Sequential # For building the neural network
from tensorflow.keras.layers import Dense, Dropout, LSTM # For building neural networks

##### Fetching the Bitcoin Price Data

In [ ]:
# Fetch Bitcoin data
df = yf.download("ETH-USD", start="2023-01-01", end="2024-11-30")

# Display the first few rows of the data
df.head()

# Plot the closing price to visualize trends
plt.figure(figsize=(12, 6))
plt.plot(df['Close'], label="Closing Price", color='green')
plt.title("Ethereum Closing Price (2024)")
plt.xlabel("Date")
plt.ylabel("Price (USD)")
plt.legend()
plt.show()

##### Data Infos

In [207]:
print("=============================================")
print("Data types of the columns in the data frame:", df.dtypes) # Data types of the columns
print("=============================================")
print("Shape of the data frame:", df.shape) # Shape of the data frame
print("=============================================")
print("Lenght of the data:", len(df)) # Length of the data
print("=============================================")
print("Checking for missing values in the data:\n", df.isnull().sum()) # Checking for missing values
print("=============================================")
df.describe() # Summary statistics of the data

Data types of the columns in the data frame: Price      Ticker 
Adj Close  ETH-USD    float64
Close      ETH-USD    float64
High       ETH-USD    float64
Low        ETH-USD    float64
Open       ETH-USD    float64
Volume     ETH-USD      int64
dtype: object
Shape of the data frame: (699, 6)
Lenght of the data: 699
Checking for missing values in the data:
 Price      Ticker 
Adj Close  ETH-USD    0
Close      ETH-USD    0
High       ETH-USD    0
Low        ETH-USD    0
Open       ETH-USD    0
Volume     ETH-USD    0
dtype: int64


Price,Adj Close,Close,High,Low,Open,Volume
Ticker,ETH-USD,ETH-USD,ETH-USD,ETH-USD,ETH-USD,ETH-USD
count,699.000000,699.000000,699.000000,699.000000,699.000000,6.990000e+02
mean,2364.854938,2364.854938,2412.035784,2311.447229,2361.463078,1.229910e+10
std,703.822843,703.822843,725.257874,678.227062,703.622940,8.436459e+09
min,1200.964844,1200.964844,1203.475342,1192.885376,1196.713623,2.081626e+09
25%,1800.300476,1800.300476,1827.601135,1769.574402,1797.976562,6.652484e+09
50%,2231.465332,2231.465332,2276.764648,2180.225342,2226.892578,1.015547e+10
75%,2972.332275,2972.332275,3034.360229,2888.783813,2969.967041,1.535598e+10
max,4066.445068,4066.445068,4092.284180,3936.627197,4066.690430,6.766813e+10


##### Data PreProcessing

In [ ]:
# Use only the 'Close' column for prediction
data = df[['Close']]

# Scale data between 0 and 1 for LSTM
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_data = scaler.fit_transform(data.values)

# Define prediction window size
prediction_days = 30  # Use the last 30 days to predict the next day

# Create training data
x_train, y_train = [], []

for i in range(prediction_days, len(scaled_data)):
    x_train.append(scaled_data[i - prediction_days:i, 0])
    y_train.append(scaled_data[i, 0])

# Convert x_train and y_train to numpy arrays
x_train, y_train = np.array(x_train), np.array(y_train)

# Reshape data for LSTM input
x_train = np.reshape(x_train, (x_train.shape[0], x_train.shape[1], 1))

##### Creating the Neural Network with LSTM

In [ ]:
# Initialize the LSTM model
model = Sequential()

# Add layers
model.add(LSTM(units=100, return_sequences=True, input_shape=(x_train.shape[1], 1)))
model.add(Dropout(0.3))  # Prevent overfitting
model.add(LSTM(units=100, return_sequences=True))
model.add(Dropout(0.3))
model.add(LSTM(units=100))
model.add(Dropout(0.3))
model.add(Dense(units=1))  # Final output layer

# Compile the model     
model.compile(optimizer='adam', loss='mean_squared_error')

# Train the model
history = model.fit(x_train, y_train, epochs=50, batch_size=32, verbose=1)

##### Model Data

In [ ]:
model.summary() # Display the model summary

##### Test Data

In [ ]:
# Fetch the last 120 days of data for prediction
future_days = 120  # Number of days to predict

# Prepare test data
test_data = df[-(prediction_days + future_days):]  # Include a small recent dataset
test_scaled = scaler.transform(test_data['Close'].values.reshape(-1, 1))

# Create x_test and y_test
x_test = []
y_test = test_scaled[prediction_days:]  # Actual prices for evaluation

# Create x_test
for i in range(prediction_days, len(test_scaled)):
    x_test.append(test_scaled[i - prediction_days:i, 0])

# Convert x_test to a numpy array
x_test = np.array(x_test)
x_test = np.reshape(x_test, (x_test.shape[0], x_test.shape[1], 1))

##### Prediction Ethereum Price

In [ ]:
# Predict Bitcoin prices
predicted_scaled = model.predict(x_test)

# Inverse scale predictions to get actual price
predicted_prices = scaler.inverse_transform(predicted_scaled)

# Inverse scale y_test for comparison
actual_prices = scaler.inverse_transform(y_test)

# Generate a date range for x-axis
dates = df.index[-len(actual_prices):]

# Plot actual vs predicted prices with dates
plt.figure(figsize=(15, 7))
plt.plot(dates, actual_prices, label="Actual Prices", color="green", linewidth=4)
plt.plot(dates, predicted_prices, label="Predicted Prices", color="red", linestyle="--", linewidth=3)
plt.title("Ethereum Price Prediction (Daily Values)")
plt.xlabel("Date")
plt.ylabel("Price (USD)")
plt.legend()
plt.grid(True)

##### Evaulating the Model
F1 score and accuracy are not suitable methods for regression models.For regression models, metrics like MSE, MAPE, and RMSE provide better insights.

In [196]:
# Convert prices to labels (1 if price goes up, 0 if price goes down)
actual_labels = np.where(actual_prices[1:] > actual_prices[:-1], 1, 0)
predicted_labels = np.where(predicted_prices[1:] > actual_prices[:-1], 1, 0)

# Calculate accuracy and F1-score
f1 = f1_score(actual_labels, predicted_labels)
accuracy = accuracy_score(actual_labels, predicted_labels)
recall_score = recall_score(actual_labels, predicted_labels)
precision_score = precision_score(actual_labels, predicted_labels)

# Print the classification report
print(f"=============================================")
print(f"Accuracy: {accuracy * 100:.2f}%")
print(f"F1-Score: {f1:.2f}")
print(f"Recall: {recall_score:.2f}")
print(f"Precision: {precision_score:.2f}")
# Detailed classification report
print(f"=============================================")
print(classification_report(actual_labels, predicted_labels))
print(f"=============================================")

Accuracy: 54.62%
F1-Score: 0.58
Recall: 0.61
Precision: 0.56
              precision    recall  f1-score   support

           0       0.53      0.47      0.50        57
           1       0.56      0.61      0.58        62

    accuracy                           0.55       119
   macro avg       0.54      0.54      0.54       119
weighted avg       0.54      0.55      0.54       119



In [198]:
# Calculate regression metrics
mse = mean_squared_error(actual_prices, predicted_prices)
mae = mean_absolute_error(actual_prices, predicted_prices)
rmse = root_mean_squared_error(actual_prices, predicted_prices)
mape = mean_absolute_percentage_error(actual_prices, predicted_prices) * 100

# Print the error metrics
print(f"=============================================")
print(f"Mean Squared Error: {mse:.2f}") # Mean Squared Error
print(f"Root Mean Squared Error: {rmse:.2f}") # Root Mean Squared Error
print(f"=============================================")
print(f"Mean Absolute Error: {mae:.2f} $ Gunluk Sapma Miktari") # Mean Absolute Error
print(f"=============================================")
print(f"Mean Absolute Percentage Error (MAPE): {mape:.2f}%") # Mean Absolute Percentage Error
print(f"! A MAPE below 10-20% is generally considered good for financial predictions.")
print(f"=============================================")

Mean Squared Error: 21808.15
Root Mean Squared Error: 147.68
Mean Absolute Error: 109.09 $ Gunluk Sapma Miktari
Mean Absolute Percentage Error (MAPE): 4.07%
! A MAPE below 10-20% is generally considered good for financial predictions.


##### Comparing the Train and Test Scores

In [ ]:
# Train and test losses
train_loss = history.history['loss'][-1]  # Final loss after training
test_loss = model.evaluate(x_test, y_test) # Loss on test data

# Print the final train and test losses
print(f"=============================================")
print(f"Train Loss: {train_loss}")
print(f"Test Loss: {test_loss}")
print(f"=============================================")